In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from azure.storage.blob import BlobServiceClient
import os

# Configurações de caminhos para camada Bronze e Silver
bronze_path = f"{os.environ['AZURE_STORAGE_ACCOUNT_URL']}/bronze"
silver_path = f"{os.environ['AZURE_STORAGE_ACCOUNT_URL']}/silver"

# Inicialização do SparkSession
spark = SparkSession.builder \
    .appName("Transformar Bronze para Silver") \
    .getOrCreate()

# Listar todos os arquivos na camada Bronze (com base em arquivos .parquet)
blob_service_client = BlobServiceClient(account_url=os.environ['AZURE_STORAGE_ACCOUNT_URL'])
container_client = blob_service_client.get_container_client("bronze")
bronze_tables = [blob.name for blob in container_client.list_blobs() if blob.name.endswith(".parquet")]

# Loop para processar cada tabela da camada Bronze
for table_path in bronze_tables:
    # Nome da tabela com base no caminho do arquivo
    table_name = table_path.split("/")[-1].replace(".parquet", "")

    # Leitura dos dados da camada Bronze
    df_bronze = spark.read.parquet(f"abfss://bronze@{os.environ['AZURE_STORAGE_ACCOUNT_NAME']}.dfs.core.windows.net/{table_path}")

    # Tratamentos básicos
    # Remover duplicatas
    # Preencher valores nulos com um padrão genérico
    # Preencher valores nulos em campos numéricos com zero
    df_silver = df_bronze \
        .dropDuplicates() \
        .na.fill("N/A") \
        .na.fill(0)

    # Normalização de tipos e colunas
    for column in df_silver.columns:
        # Exemplo de normalização: converter todas as strings para minúsculas
        if df_silver.schema[column].dataType.simpleString() == "string":
            df_silver = df_silver.withColumn(column, col(column).alias(column.lower()))

    # Caminho da tabela transformada na camada Silver
    silver_table_path = f"{silver_path}/{table_name}.parquet"

    # Gravação no Azure Data Lake em formato Parquet
    df_silver.write \
        .mode("overwrite") \
        .parquet(f"abfss://silver@{os.environ['AZURE_STORAGE_ACCOUNT_NAME']}.dfs.core.windows.net/{silver_table_path}")

    print(f"Tabela {table_name} transformada e salva em {silver_table_path}")

# Encerrar sessão Spark
spark.stop()


ClientAuthenticationError: Server failed to authenticate the request. Please refer to the information in the www-authenticate header.
RequestId:8d131a00-301e-0022-3b75-403ca1000000
Time:2024-11-27T02:41:12.8593756Z
ErrorCode:NoAuthenticationInformation
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>NoAuthenticationInformation</Code><Message>Server failed to authenticate the request. Please refer to the information in the www-authenticate header.
RequestId:8d131a00-301e-0022-3b75-403ca1000000
Time:2024-11-27T02:41:12.8593756Z</Message></Error>